In [6]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from util import get_data
from marketsimcode import compute_portvals

In [7]:
def Portfolio_Statistics(portvals):
    daily_returns = portvals.copy()
    daily_returns[1:] = (portvals[1:] / portvals[:-1].values) - 1
    daily_returns = daily_returns[1:]

    cum_ret = (portvals.iloc[-1] / portvals.iloc[0]) - 1
    avg_daily_ret = daily_returns.mean()
    std_daily_ret = daily_returns.std()
    sharpe_ratio = np.sqrt(252.0) * (avg_daily_ret / std_daily_ret)

    return cum_ret,avg_daily_ret,std_daily_ret,sharpe_ratio


In [8]:
def benchmark(symbol, sd, ed, shares):  

    prices = get_data([symbol], pd.date_range(sd,ed))
    if 'SPY' not in symbol:
            prices.drop('SPY', axis = 1, inplace = True)
         
    prices= prices.fillna(method ='ffill')
    prices = prices.fillna(method = 'bfill')
    prices = prices/prices.iloc[0]

    df_trades = pd.DataFrame(index = prices.index)
    df_trades[symbol] = 0

    start =df_trades.index.min()
    end = df_trades.index.max()
    df_trades.loc[start,symbol] = shares
    df_trades.loc[end,symbol] = -1*shares
    return df_trades

In [9]:

def get_df(prices, symbol):
    df= prices[symbol]
    df= df.fillna(method= 'ffill').fillna(method= 'bfill')
    df= df/df.iloc[0]
    df.columns = ["Price"]
    return df

def get_SMA(prices,symbol, window=10):
    df= get_df(prices, symbol)
    rm= df["Price"].rolling(window).mean()
    pdsma= df.divide(rm, axis=0)
    return rm, pdsma

def get_BB(prices, symbol, window=10):
    df= get_df(prices, symbol)
    rm= df["Price"].rolling(window).mean()
    rstd= df["Price"].rolling(window).std()
    upper= rm + rstd*2
    lower= rm - rstd*2
    return upper, lower

def get_Momentum(prices, symbol, window=10):
    df= get_df(prices, symbol)
    momentum= df["Price"]/df["Price"].shift(window) - 1
    return momentum


In [8]:
def testPolicy(symbol = "AAPL", sd = dt.datetime(2010, 1, 1), ed = dt.datetime(2011,12,31), sv = 100000):
    
    signal = 0  #can be -1, 0, or 1, corresponding to a “short,” “out” or “long” position
    moving_window = 20
    prices = get_data([symbol], pd.date_range(sd, ed))
    
    if 'SPY' not in symbol:
            prices.drop('SPY', axis = 1, inplace = True)
    prices= prices.fillna(method ='ffill')
    prices = prices.fillna(method = 'bfill')
    prices = prices/prices.iloc[0,:]
    
    
    trades_df = pd.DataFrame(columns=['Order', 'Shares'], index = prices.index)
    
    sma, price_sma = getsma(prices, moving_window)

    upperband, lowerband, BBP = getBBP(prices, moving_window) 
    momentum = getMomentum(prices, moving_window)
   
    print('came here') 
    
    for index in range(0, prices.shape[0]-1):
        i = prices.index[index]
        if signal == 0:
            if BBP.loc[i, symbol] < 0.2 or price_sma.loc[i,symbol] < 0.6 or momentum.loc[i,symbol] < -0.1:
                trades_df.loc[i] = [ 'BUY', 1000]
                signal = 1

            elif BBP.loc[i,symbol] > 0.8 or price_sma.loc[i,symbol] > 1.0 or momentum.loc[i,symbol] > 0.1:
                trades_df.loc[i] = ['SELL', 1000]
                signal = -1

        elif signal == 1:
            if BBP.loc[i, symbol] > 0.9 or price_sma.loc[i,symbol] > 1.5 or momentum.loc[i,symbol] > 0.2:
                trades_df.loc[i] = [ 'SELL', 2000]
                signal = -1

            elif BBP.loc[i, symbol] > 0.8 or price_sma.loc[i,symbol] > 1.1 or momentum.loc[i,symbol] >0.1:
                trades_df.loc[i] = [ 'SELL', 1000]
                singal = 0

        elif signal == -1:
            if BBP.loc[i, symbol]  < 0.1 or price_sma.loc[i,symbol] < 0.7 or momentum.loc[i,symbol] < -0.2:
                trades_df.loc[i] = ['BUY', 2000]
                signal = 1

            elif BBP.loc[i, symbol]  < 0.2 or price_sma.loc[i,symbol] < 0.6 or momentum.loc[i,symbol] < -0.1:
                trades_df.loc[i] = ['BUY', 1000]
                signal = 0

    if (signal == 1):
        trades_df.loc[i] = ['SELL', 1000]
    else:
        trades_df.loc[i] = ['BUY', 1000]

    trades_df['Shares'] = np.where(trades_df['Order'] =='BUY', trades_df['Shares'], -1 * trades_df['Shares'])
    trades_df = trades_df.loc[:,['Shares']]
    trades_df.columns = [symbol]
    trades_df.fillna(0, inplace =True)
        
    return trades_df
    
   

    
    

In [10]:
def testpolicy(symbol = "AAPL", sd = dt.datetime(2010, 1, 1), ed = dt.datetime(2011,12,31), sv = 100000):
    
    signal = 0  #can be -1, 0, or 1, corresponding to a “short,” “out” or “long” position
    moving_window = 20
    prices = get_data([symbol], pd.date_range(sd, ed))
    
    if 'SPY' not in symbol:
            prices.drop('SPY', axis = 1, inplace = True)
    prices= prices.fillna(method ='ffill')
    prices = prices.fillna(method = 'bfill')
    prices = prices/prices.iloc[0,:]
    
    
    trades_df = pd.DataFrame(columns=['Order', 'Shares'], index = prices.index)
    
    sma, pdsma = getsma(prices, moving_window)

    upper, lowerband, BBP = getBBP(prices, moving_window) 
    momentum = getMomentum(prices, moving_window)
   
    print('came here') 
    shares=0
    trades = pd.DataFrame(columns=['Order', 'Shares'])
   
    for index in range(0, prices.shape[0]-1):
        i = prices.index[index]
        
        if(shares == 1000 and (pdsma.loc[i,symbol]> 1.4 or upper.loc[i,symbol]> 0.8 or momentum.loc[i,symbol]> 0.2)):
            trades.loc[i] = ['SELL',2000]
            shares= -1000
        elif(shares == 1000 and (pdsma.loc[i,symbol]> 1.0 or upper.loc[i,symbol]> 0.7 or momentum.loc[i,symbol]> 0.1)):
            trades.loc[i] = ['SELL',1000]
            shares= 0
        elif(shares == -1000 and (pdsma.loc[i,symbol]< 0.7 or upper.loc[i,symbol]< 0.2 or momentum.loc[i,symbol]< -0.2)):
            trades.loc[i] = ['BUY',2000]
            shares= 1000
        elif(shares == -1000 and (pdsma.loc[i,symbol]< 0.6 or upper.loc[i,symbol]< 0.3 or momentum.loc[i,symbol]< -0.1)):
            trades.loc[i] = ['BUY',1000]
            shares= 0
        elif(shares == 0 and (pdsma.loc[i,symbol]< 0.6  or upper.loc[i,symbol]< 0.3 or momentum.loc[i,symbol]< -0.1)):
            trades.loc[i] = ['BUY',1000]
            shares= 1000
        elif(shares == 0 and (pdsma.loc[i,symbol]> 1.0 or upper.loc[i,symbol]> 0.7 or momentum.loc[i,symbol]> 0.1)):
            trades.loc[i] = ['SELL',1000]
            shares= -1000

    last = prices.index[prices.shape[0]-1] 

    if(shares==1000):
        trades.loc[last] = ['SELL',1000]
    if(shares == -1000):
        trades.loc[last] = ['BUY',1000]

    trades['Shares'] = np.where(trades['Order'] =='BUY', trades['Shares'], -1 * trades['Shares'])
    trades = trades.loc[:,['Shares']]
    trades.columns = [symbol]
    trades.fillna(0, inplace =True)
        
    return trades
    
   

    
    

In [39]:
symbol = "JPM"
sd = dt.datetime(2008, 1, 1)
ed = dt.datetime(2009,12,31)
sv = 100000
    
signal = 0  #can be -1, 0, or 1, corresponding to a “short,” “out” or “long” position
moving_window = 20
prices = get_data([symbol], pd.date_range(sd, ed))

if 'SPY' not in symbol:
        prices.drop('SPY', axis = 1, inplace = True)
prices= prices.fillna(method ='ffill')
prices = prices.fillna(method = 'bfill')
prices = prices/prices.iloc[0,:]


trades_df = pd.DataFrame(columns=['Order', 'Shares'], index = prices.index)

sma, pdsma = getsma(prices, moving_window)

upper, lowerband, BBP = getBBP(prices, moving_window) 
momentum = getMomentum(prices, moving_window)

print('came here') 
shares=0
trades = pd.DataFrame(columns=['Order', 'Shares'])

for index in range(0, prices.shape[0]-1):
    i = prices.index[index]

    if(shares == 1000 and (pdsma.loc[i,symbol]> 1.4 or upper.loc[i,symbol]> 0.8 or momentum.loc[i,symbol]> 0.2)):
        trades.loc[i] = ['SELL',2000]
        shares= -1000
    elif(shares == 1000 and (pdsma.loc[i,symbol]> 1.0 or upper.loc[i,symbol]> 0.7 or momentum.loc[i,symbol]> 0.1)):
        trades.loc[i] = ['SELL',1000]
        shares= 0
    elif(shares == -1000 and (pdsma.loc[i,symbol]< 0.7 or upper.loc[i,symbol]< 0.2 or momentum.loc[i,symbol]< -0.2)):
        trades.loc[i] = ['BUY',2000]
        shares= 1000
    elif(shares == -1000 and (pdsma.loc[i,symbol]< 0.6 or upper.loc[i,symbol]< 0.3 or momentum.loc[i,symbol]< -0.1)):
        trades.loc[i] = ['BUY',1000]
        shares= 0
    elif(shares == 0 and (pdsma.loc[i,symbol]< 0.6  or upper.loc[i,symbol]< 0.3 or momentum.loc[i,symbol]< -0.1)):
        trades.loc[i] = ['BUY',1000]
        shares= 1000
    elif(shares == 0 and (pdsma.loc[i,symbol]> 1.0 or upper.loc[i,symbol]> 0.7 or momentum.loc[i,symbol]> 0.1)):
        trades.loc[i] = ['SELL',1000]
        shares= -1000

last = prices.index[prices.shape[0]-1] 

if(shares==1000):
    trades.loc[last] = ['SELL',1000]
if(shares == -1000):
    trades.loc[last] = ['BUY',1000]

trades['Shares'] = np.where(trades['Order'] =='BUY', trades['Shares'], -1 * trades['Shares'])
trades = trades.loc[:,['Shares']]
trades.columns = [symbol]
trades.fillna(0, inplace =True)





    
    

came here


In [4]:
def test_code():
    start_date = dt.datetime(2008,1,1)
    end_date = dt.datetime(2009,12,31)
    symbol = "JPM"
    moving_window = 20
    
    
    #Manual Strategy
    df_trades_manual = testpolicy(symbol = "JPM", sd=start_date, ed=end_date, sv = 100000)
    port_vals_manual = compute_portvals(df_trades_manual, start_val = 100000, commission = 9.95, impact = 0.005)
    
    #Normalizing PortFolio Values
    port_vals_norm_manual = port_vals_manual/port_vals_manual.iloc[0]

    cum_ret_man,avg_daily_ret_man,std_daily_ret_man,sharpe_ratio_man = Portfolio_Statistics(port_vals_norm_manual)
    
    print(f"Date Range: {start_date} to {end_date} for {symbol}")
    print()
    print("Manual Strategy")
    print(f"Sharpe Ratio of Fund: {sharpe_ratio_man}")  
    print(f"Cumulative Return of Fund: {cum_ret_man}") 
    print(f"Standard Deviation of Fund: {std_daily_ret_man}")
    print(f"Average Daily Return of Fund: {avg_daily_ret_man}") 
    print(f"Final Portfolio Value: {port_vals_manual[-1]}")
    
    
    #Benchmark Strategy 
    df_trades_benchmark = benchmark(symbol = "JPM", sd=start_date, ed= end_date,shares = 1000)
    port_vals_benchmark = compute_portvals(df_trades_benchmark, start_val = 100000, commission = 9.95, impact = 0.005)
    
    #Normalizing PortFolio Values
    port_vals_norm_benchmark = port_vals_benchmark/port_vals_benchmark.iloc[0]

    cum_ret_bench,avg_daily_ret_bench,std_daily_ret_bench,sharpe_ratio_bench = Portfolio_Statistics(port_vals_norm_benchmark)
    print()
    print("Benchmark Strategy")

    print(f"Sharpe Ratio of Fund: {sharpe_ratio_bench}")  
    print(f"Cumulative Return of Fund: {cum_ret_bench}") 
    print(f"Standard Deviation of Fund: {std_daily_ret_bench}")
    print(f"Average Daily Return of Fund: {avg_daily_ret_bench}") 
    print(f"Final Portfolio Value: {port_vals_benchmark[-1]}")
    

    plt.figure(figsize=(15,7))
    plt.plot(port_vals_norm_manual, 'red')
    plt.plot(port_vals_norm_benchmark,'green')
    plt.title("Portfolio vs Benchmark")
    plt.show()


In [5]:
if __name__ == '__main__':
    test_code()
   

NameError: name 'dt' is not defined